In [3]:
from sklearn.datasets import load_boston
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np

In [4]:
#Dessa vez, o dataset veio direto do pacote do sklearn 
Boston = load_boston()

C:\Users\jllce\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i

In [5]:
df = pd.DataFrame(Boston.data, columns=Boston.feature_names)
df["Price"] = pd.Series(Boston.target)

X = df.iloc[:,0:13]
y = df.iloc[:,-1]

In [8]:
#instanciando e criando os parâmetros do Seletor Sequencial de Features

sfs = SFS(LinearRegression(), # Aqui eu passo qual o modelo que usarei. No caso, usaremos a Regressão Linear
         k_features = 5, # qtd de features que quero que seja retornado
         forward=False, #muito importante definir esse parametro igual a false, pois assim eu digo que
                        #quero usar o backward feature eliminatione não o forward. O backward é basicamente fazer
                        #comparações de valores entre o valor p e a significancia da variavel para nao termos uma 
                        #variancia tao discrepante assim
         floating=False,
         scoring = "r2", #utulizarei para o score a métrica do r-quadrado, mas poderia ser tambem o qui-quadrado
                         #ou outros estimadores
         cv=0) #Estamos buscando fazer o backward feature elimination, mas não farei validação cruzada com ninguem.
               #pois, os dados que eu quero serão somente aqueles presentes na variável treino
sfs.fit(X,y)

df_SFS_results = pd.DataFrame(sfs.subsets_).transpose()

#Veremos o resultado da seleção de features através da eliminação. Então, ele começa apurando o score com todas as features,
# depois elimina a pior e mostra como o score ficou, e assim sucessivamente até chegar ao número que eu solicitei (5)
df_SFS_results

,feature_idx,cv_scores,avg_score,feature_names
13,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)",[0.7406426641094094],0.740643,"(CRIM, ZN, INDUS, CHAS, NOX, RM, AGE, DIS, RAD..."
12,"(0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12)",[0.7406412165505145],0.740641,"(CRIM, ZN, INDUS, CHAS, NOX, RM, DIS, RAD, TAX..."
11,"(0, 1, 3, 4, 5, 7, 8, 9, 10, 11, 12)",[0.7405822802569575],0.740582,"(CRIM, ZN, CHAS, NOX, RM, DIS, RAD, TAX, PTRAT..."
10,"(0, 1, 4, 5, 7, 8, 9, 10, 11, 12)",[0.7352631473231817],0.735263,"(CRIM, ZN, NOX, RM, DIS, RAD, TAX, PTRATIO, B,..."
9,"(0, 4, 5, 7, 8, 9, 10, 11, 12)",[0.7292543470977955],0.729254,"(CRIM, NOX, RM, DIS, RAD, TAX, PTRATIO, B, LSTAT)"
8,"(0, 4, 5, 7, 8, 10, 11, 12)",[0.7239765998018792],0.723977,"(CRIM, NOX, RM, DIS, RAD, PTRATIO, B, LSTAT)"
7,"(4, 5, 7, 8, 10, 11, 12)",[0.7187395846343028],0.71874,"(NOX, RM, DIS, RAD, PTRATIO, B, LSTAT)"
6,"(4, 5, 7, 10, 11, 12)",[0.7153894128095097],0.715389,"(NOX, RM, DIS, PTRATIO, B, LSTAT)"
5,"(4, 5, 7, 10, 12)",[0.7080892893529662],0.708089,"(NOX, RM, DIS, PTRATIO, LSTAT)"


In [9]:
#Para o caso de usarmos a Forward Feature Selection, o caminho será inverso. Usaremos praticamente toda a
# estrutura do backward feature elimination, mas mudaremos o forward pra true.

sfs = SFS(LinearRegression(),
          k_features = 5,
         forward=True,
         floating=False,
         scoring = "r2", 
         cv=0) #o cross validation significa que não estou fazendo validação cruzada com nada além dos meus dados de treino

sfs.fit(X,y)

df_SFS_results = pd.DataFrame(sfs.subsets_).transpose()

# Enfim, veremos que é basicamente o caminho contrário do backward. Pode-se utilizar qualquer uma, o que vai depender é o
# tipo de código que estou implementando. Talvez o tempo de execução possa fazer a diferença.
# Então, basicamente a diferença de uso é saber se faz mais sentido em alguma dataset eu trabalhar pegando todas as 
#features e ir eliminando uma a uma pra chegar onde quero, ou se vale pegar uma feature e ir adicionando até chegar em 
#um ponto que seja válido pra mim.
df_SFS_results

,feature_idx,cv_scores,avg_score,feature_names
1,"(12,)",[0.5441462975864799],0.544146,"(LSTAT,)"
2,"(5, 12)",[0.6385616062603403],0.638562,"(RM, LSTAT)"
3,"(5, 10, 12)",[0.6786241601613112],0.678624,"(RM, PTRATIO, LSTAT)"
4,"(5, 7, 10, 12)",[0.6903077016842538],0.690308,"(RM, DIS, PTRATIO, LSTAT)"
5,"(4, 5, 7, 10, 12)",[0.7080892893529662],0.708089,"(NOX, RM, DIS, PTRATIO, LSTAT)"
